In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import models, transforms, datasets
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, precision_score, f1_score, recall_score

from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
import numpy as np
import itertools
from tqdm import tqdm
import torch.optim.lr_scheduler as lr_scheduler
import warnings
import pandas as pd
from early_stopping import EarlyStopping
import multiprocessing
# Suppress specific warnings
warnings.filterwarnings("ignore", message="Mean of empty slice")
warnings.filterwarnings("ignore", message="invalid value encountered in scalar divide")


2024-02-22 23:26:52.246086: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-22 23:26:52.246121: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-22 23:26:52.346484: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-22 23:26:52.488292: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
!git commit - 'Komondor test

In [ ]:
torch.hub.set_dir('../models')

In [ ]:
def get_lr_scheduler(scheduler_name, optimizer, **kwargs):
    """
    Function to get a specific learning rate scheduler instance.

    Args:
        scheduler_name (str): Name of the desired scheduler type.
        optimizer (torch.optim.Optimizer): Optimizer instance.
        **kwargs: Additional keyword arguments to be passed to the scheduler constructor.

    Returns:
        torch.optim.lr_scheduler._LRScheduler: Instance of the specified learning rate scheduler.
    """
    if scheduler_name == 'StepLR':
        return lr_scheduler.StepLR(optimizer, **kwargs)
    elif scheduler_name == 'MultiStepLR':
        return lr_scheduler.MultiStepLR(optimizer, **kwargs)
    elif scheduler_name == 'ExponentialLR':
        return lr_scheduler.ExponentialLR(optimizer, **kwargs)
    elif scheduler_name == 'CosineAnnealingLR':
        return lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, **kwargs)
    elif scheduler_name == 'ReduceLROnPlateau':
        return lr_scheduler.ReduceLROnPlateau(optimizer, **kwargs)
    else:
        raise ValueError(f"Unsupported scheduler name: {scheduler_name}")

In [ ]:
def calculate_metrics(epoch,metric_type, loss, labels, predictions):
    return {
        "epoch": [epoch],
        "metric_type": [metric_type],
        "accuracy": [accuracy_score(labels, predictions)],
        "precision": [precision_score(labels, predictions, average="weighted")],
        "recall": [recall_score(labels, predictions, average="weighted")],
        "f1": [f1_score(labels, predictions, average="weighted")],
        "loss" : [loss]
    }

def log_metrics(writer, metrics):
    # Log metrics to TensorBoard
    for key, value in metrics.items():
        if key != "epoch" and key != "metric_type":  # Exclude non-numeric values
            writer.add_scalar(f"{metrics['metric_type'][0]}/{key}", value[0], metrics['epoch'][0])

In [8]:
args = {
    'epoch': 30,
    'train_batch_size' : 256,
    'val_batch_size' : 128,
    'num_workers': 8,
    'lr' : 0.001,
    'adam_eps' : 1e-8,
    'adam_beta1' : 0.9,
    'adam_beta2' : 0.999,
    'adam_weight_decay': 0.01,
    'lr_scheduler_type': 'CosineAnnealingLR',

    'es_patience': 3,
    'es_delta': 0,
}



In [17]:


# Get the number of available CPU cores
num_cores = multiprocessing.cpu_count()
num_cores


256

In [9]:
# Define transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [18]:
#  Load datasets
train_dataset = datasets.ImageFolder('../data/kaggle_brain_tumor/Training', transform=train_transform)
test_dataset = datasets.ImageFolder('../data/kaggle_brain_tumor/Testing', transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=args['train_batch_size'], shuffle=True, num_workers=16)
test_loader = DataLoader(test_dataset, batch_size=args['val_batch_size'], shuffle=False)

/opt/conda/lib/python3.11/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 256 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [11]:


def train_model(model, criterion, optimizer, es_checker:EarlyStopping, num_epochs, device):
    # Tensorboard writer
    writer = SummaryWriter()    
    metrics_df = pd.DataFrame(columns=['epoch', 'metric_type', 'loss', "accuracy", "precision", "recall", "f1"])
     
    for epoch in range(num_epochs):
        
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        all_train_predictions = []
        all_train_labels = []

        # Wrap train_loader with tqdm for progress visualization
        train_loader_with_progress = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)

        for inputs, labels in train_loader_with_progress:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            
            all_train_predictions.extend(predicted.cpu().numpy())
            all_train_labels.extend(labels.cpu().numpy())

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            train_loader_with_progress.set_postfix({'loss': loss.item(), 'acc': (predicted == labels).sum().item() / labels.size(0)})

        train_loss = running_loss / len(train_loader)
        train_metrics = calculate_metrics(epoch, 'train', train_loss, all_train_labels, all_train_predictions)

        new_df = pd.DataFrame.from_dict(train_metrics)
        metrics_df = pd.concat([metrics_df, new_df], ignore_index=True)


        log_metrics(writer, train_metrics)

        # Validation
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        all_val_predictions = []
        all_val_labels = []
        model.eval()
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

                all_val_predictions.extend(predicted.cpu().numpy())
                all_val_labels.extend(labels.cpu().numpy())
            

            val_loss = val_loss / len(test_loader)
                

        val_metrics = calculate_metrics(epoch, 'val', val_loss, all_val_labels, all_val_predictions)

        new_df = pd.DataFrame.from_dict(val_metrics)
        metrics_df = pd.concat([metrics_df, new_df], ignore_index=True)

        log_metrics(writer, val_metrics)

        print(f"Epoch {epoch+1}/{num_epochs}")

        es_checker(val_loss, model)
        if es_checker.early_stop:
            print("Early stopping")
            break
        
    # load the last checkpoint with the best model
    model.load_state_dict(torch.load('best_checkpoint.pt'))
    torch.save(model.state_dict(), 'last_checkpoint.pt')
    writer.close()
    return model, metrics_df


In [9]:
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)
scheduler = get_lr_scheduler(args["lr_scheduler_type"], optimizer)
es_checker = EarlyStopping(args["es_patience"], True, args["es_delta"], path="best_checkpoint.pt")

/opt/conda/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
# Train the model
results = train_model(model, criterion, optimizer, es_checker, num_epochs=args['epoch'], device=device)

/tmp/slurm-3904980/ipykernel_524178/1803605216.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_df = pd.concat([metrics_df, new_df], ignore_index=True)


Epoch 1/30
Validation loss decreased (inf --> 0.812106).  Saving model ...


Epoch 2/30
Validation loss decreased (0.812106 --> 0.686264).  Saving model ...


Epoch 3/30
Validation loss decreased (0.686264 --> 0.515801).  Saving model ...


Epoch 4/30
Validation loss decreased (0.515801 --> 0.460734).  Saving model ...


...

Epoch 27/30
EarlyStopping counter: 1 out of 3


Epoch 28/30
Validation loss decreased (0.290011 --> 0.274961).  Saving model ...


Epoch 29/30
EarlyStopping counter: 1 out of 3


Epoch 30/30
EarlyStopping counter: 2 out of 3


FileNotFoundError: [Errno 2] No such file or directory: 'checkpoint.pt'

In [2]:
results.head()

NameError: name 'results' is not defined

In [ ]:
train_res = results[results.metric_type == 'train']
val_res = results[results.metric_type == 'val']

In [ ]:
plt.plot(train_res['epoch'],train_res['accuracy'], label='train_accuracy')
plt.plot(val_res['epoch'],val_res['accuracy'], label='val_accuracy')
plt.legend()
plt.show()

In [ ]:
plt.plot(train_res['epoch'],train_res['loss'], label='train_loss')
plt.plot(val_res['epoch'],val_res['loss'], label='val_loss')
plt.legend()
plt.show()

## Full fine-tune

In [20]:
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(train_dataset.classes))

model.load_state_dict(torch.load('best_checkpoint.pt'))
for param in model.parameters():
    param.requires_grad = True


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0001)
scheduler = get_lr_scheduler(args["lr_scheduler_type"], optimizer)
es_checker = EarlyStopping(args["es_patience"], True, args["es_delta"], path="best_full_checkpoint.pt")

In [ ]:
# Train the model
results = train_model(model, criterion, optimizer, es_checker, num_epochs=args['epoch'], device=device)

Epoch 1/30:   0%|          | 0/23 [00:00<?, ?it/s]/opt/conda/lib/python3.11/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 256 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [1]:
# Confusion matrix
def plot_confusion_matrix(y_true, y_pred, classes):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [ ]:


# Get predictions for test set
model.eval()
all_predictions = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_predictions)
print(f"Test Accuracy: {accuracy}")

# Plot confusion matrix
plot_confusion_matrix(all_labels, all_predictions, classes=test_dataset.classes)

# Show some results
# You can print some images with their predicted and true labels to visually inspect the performance of the model


In [ ]:
import pandas as pd
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Assuming you have defined your model, criterion, and test_loader previously
writer = SummaryWriter('runs')
for i in range(5):
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        all_predictions = []
        all_labels = []

        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

        val_loss /= len(test_loader)
        val_accuracy = val_correct / val_total

    writer.add_scalar('val_loss', val_loss, i)
    #writer.add_scalar('Train/Accuracy', train_accuracy, epoch)

# # Calculate metrics
# metrics = calculate_metrics(all_labels, all_predictions)
# metrics['loss'] = val_loss

# # Convert metrics to DataFrame
# metrics_df = pd.DataFrame(metrics, index=[0])

# print(metrics_df)


In [ ]:
writer = SummaryWriter('runs')
for i in range(10):
    writer.add_scalar('alma', i, i)

writer.flush()
writer.close() 

In [ ]:
%tensorboard 

In [ ]:
import os
os.listdir()

In [ ]:
writer = SummaryWriter('asd')
x = torch.arange(-5, 5, 0.1).view(-1, 1)
y = -5 * x + 0.1 * torch.randn(x.size())

model = torch.nn.Linear(1, 1)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1)

def train_model(iter):
    for epoch in range(iter):
        y1 = model(x)
        loss = criterion(y1, y)
        writer.add_scalar("Loss/train", loss, epoch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

train_model(10)
writer.flush()